In [2]:
from langchain_community.utilities import SQLDatabase
from langchain.sql_database import SQLDatabase
from langchain_groq import ChatGroq
import psycopg2
import os
from dotenv import load_dotenv

In [3]:
load_dotenv()

api_key = os.getenv('API_KEY')
database_url = os.getenv('DATABASE_URL')

In [4]:
# Setting the PostgreSQL URI using the loaded database URL
POSTGRES_URI = database_url

In [5]:
# Initializing the SQLDatabase utility for LangChain with the PostgreSQL URI
sql_db = SQLDatabase.from_uri(POSTGRES_URI)

In [ ]:
print(sql_db.dialect)

In [ ]:
print(sql_db.get_usable_table_names())

In [8]:
# Setting up the ChatGroq LLM with parameters and the API key
llm = ChatGroq(
    model="mixtral-8x7b-32768",
    api_key=api_key,  
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [9]:
from typing_extensions import TypedDict

# Defining the structure of the application state
class State(TypedDict):
    question: str
    query: str
    result: str
    answer: str

In [ ]:
from langchain import hub

# Pulling a pre-defined prompt template for SQL query generation
query_prompt_template = hub.pull("langchain-ai/sql-query-system-prompt")

assert len(query_prompt_template.messages) == 1
query_prompt_template.messages[0].pretty_print()

In [11]:
from typing_extensions import Annotated


class QueryOutput(TypedDict):

    query: Annotated[str, ..., "Syntactically valid SQL query."]


# Function to generate SQL queries using the LLM
def write_query(state: State):
    
    prompt = query_prompt_template.invoke(
        {
            "dialect": sql_db.dialect,
            "top_k": 10,
            "table_info": sql_db.get_table_info(),
            "input": state["question"],
        }
    )
    
    structured_llm = llm.with_structured_output(QueryOutput)
    result = structured_llm.invoke(prompt)
    return {"query": result["query"]}

In [ ]:
sql_db.get_table_info()

In [12]:
# Connecting to PostgreSQL
def connect_to_db():
    conn = psycopg2.connect(POSTGRES_URI)  
    return conn

In [13]:
# Executing the SQL query
def execute_sql_query(query: str):
    conn = connect_to_db()
    cursor = conn.cursor()
    cursor.execute(query)
    result = cursor.fetchall()
    conn.close()
    return result

In [14]:
# Function to handle user questions, generate and execute SQL queries, and return answers
def query_agent(question: str):

    # Initialize state
    state = State(question=question, query="", result="", answer="")
    
    # Generating SQL query using the LLM
    query_info = write_query(state)
    query = query_info['query']
    
    # Executing the query
    result = execute_sql_query(query)
    
    # Update state with the query
    state["query"] = query

    # Store the result in the state
    state["result"] = str(result)
    
    state["answer"] = f"The result is: {result}"
    
    return state["answer"]

In [15]:
user_input = "what is the location id of Sagasahi Iron Ore Mine"  # Example question

In [ ]:
print("User Input:", user_input)

In [ ]:
write_query({"question": "what is the location id of Sagasahi Iron Ore Mine"})

In [18]:
response = query_agent(user_input)

In [ ]:
print("Agent Response:", response)

In [20]:
user_input = "what are the total number of mines"  # Example question

In [ ]:
print("User Input:", user_input)

In [ ]:
write_query({"question": "what are the total number of mines"})

In [23]:
response = query_agent(user_input)

In [ ]:
print("Agent Response:", response)